In [1]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 연습용 MNIST 데이터 셋을 읽어오는 작업

train_data = datasets.FashionMNIST(
                                    root = 'data',
                                    train = True,
                                    download = True,
                                    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
                                    root = 'data',
                                    train = False,
                                    download = True,
                                    transform = ToTensor()
)

### 데이터 구성 확인

In [8]:
print(train_data)
print()

print('type(train_data[0]) = ', type(train_data[0]))
print('len(train_data[0]) = ', len(train_data[0]))
print()

print(train_data[0][0].shape) # image data
print(train_data[0][1]) # label

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

type(train_data[0]) =  <class 'tuple'>
len(train_data[0]) =  2

torch.Size([1, 28, 28])
9


### 데이터 로드 생성

In [18]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break
    
    
'''
batch_size = 한 번의 학습 때 사용할 데이터 갯수

즉, batch_size = 64로 설정한다는 말은
60000개의 샘플을 64개 단위로 나누어, 64개 데이터의 한 묶음을 한 번의 학습 데이터로 사용한다는 의미

그래서 dataloader를 구성하는 데이터의 크기가 (64 * 1 * 28 * 28)로 나타나는 것
'''

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


'\nbatch_size = 한 번의 학습 때 사용할 데이터 갯수\n\n즉, batch_size = 64로 설정한다는 말은\n60000개의 샘플을 64개 단위로 나누어, 64개 데이터의 한 묶음을 한 번의 학습 데이터로 사용한다는 의미\n\n그래서 dataloader를 구성하는 데이터의 크기가 (64 * 1 * 28 * 28)로 나타나는 것\n'

### 모델 생성

In [25]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu' # gpu 사용을 위한 셋팅
device = 'cpu'

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
                                                nn.Linear(28 * 28, 512), # 28 * 28 형태의 데이터를 1 * 512 의 형태(선형)로 변경
                                                nn.ReLU(),
                                                nn.Linear(512, 512),
                                                nn.ReLU(),
                                                nn.Linear(512, 10) # 최종 결과물은 0 ~ 9 까지의 데이터 레이블이 출력 되어야 한다.
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        
        return logits
    
model = NeuralNetwork().to('cpu')

In [26]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 모델 최적화

In [27]:
# 손실함수 - 크로스 엔트로피에 대한 정확한 이해
loss_fn = nn.CrossEntropyLoss()

# 최적화함수
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [28]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')
            

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    
    print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

In [29]:
epochs = 5

for t in range(epochs):
    print(f'Epoch {t+1}\n------------------------------------')
    
    _ = train(train_dataloader, model, loss_fn, optimizer)
    _ = test(test_dataloader, model, loss_fn)
    
print('Done')

Epoch 1
------------------------------------
loss: 2.300001 [    0/60000]
loss: 2.290669 [ 6400/60000]
loss: 2.269254 [12800/60000]
loss: 2.262628 [19200/60000]
loss: 2.239910 [25600/60000]
loss: 2.217842 [32000/60000]
loss: 2.219355 [38400/60000]
loss: 2.187773 [44800/60000]
loss: 2.183437 [51200/60000]
loss: 2.155368 [57600/60000]
Test Error: 
 Accuracy: 54.8%, Avg loss: 2.146901

Epoch 2
------------------------------------
loss: 2.155793 [    0/60000]
loss: 2.146687 [ 6400/60000]
loss: 2.083495 [12800/60000]
loss: 2.099436 [19200/60000]
loss: 2.038959 [25600/60000]
loss: 1.986870 [32000/60000]
loss: 2.002206 [38400/60000]
loss: 1.920437 [44800/60000]
loss: 1.927192 [51200/60000]
loss: 1.855193 [57600/60000]
Test Error: 
 Accuracy: 58.4%, Avg loss: 1.852862

Epoch 3
------------------------------------
loss: 1.882458 [    0/60000]
loss: 1.858453 [ 6400/60000]
loss: 1.732945 [12800/60000]
loss: 1.778489 [19200/60000]
loss: 1.667005 [25600/60000]
loss: 1.621253 [32000/60000]
loss: 1.6

### 모델 저장하기

In [32]:
torch.save(model.state_dict(), './data/pytorch_basic_model.pth')

print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


### 모델 불러오기

In [33]:
model = NeuralNetwork()
model.load_state_dict(torch.load('./data/pytorch_basic_model.pth'))

<All keys matched successfully>

In [34]:
classes = [
            'T-shirt/top',
            'Trouser',
            'Pullover',
            'Dress',
            'Coat',
            'Sandal',
            'Shirt',
            'Sneakers',
            'Bag',
            'Ankle boot',
]

model.eval()

x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    
    print(f'Predicted: "{predicted}", Acutal: "{actual}"')

Predicted: "Ankle boot", Acutal: "Ankle boot"
